# LLM - Grands Modèles de Langage

Les [transformers](https://arxiv.org/abs/1706.03762) ont changé les paradigmes du traitement du langage naturel. Il existe de nombreux modèles anglais et multilingues, mais peu de modèles français, l'inconvénient des modèles multilingues étant principalement leur taille, et accessoirement les faiblesses de la part du corpus d'entrainement dédiée au français. Nous avons donc souhaité rendre ici hommage aux rares modèles dédiés au français, que sont [Camembert](https://huggingface.co/almanach), [Croissant](https://huggingface.co/croissantllm), [Flaubert](https://huggingface.co/flaubert), [Vigogne](https://huggingface.co/bofenghuang), et bientôt Lucie.

Les modèles multilangues comme [Gemini](../tools/gemini.ipynb) et [OpenAI](../tools/openai.ipynb), offrent des résultats supérieurs aux modèles entraînés en français seulement. Il peut néanmoins être intéressant d'utiliser ces petits modèles français quand les ressources computationnelles (CPU, GPU, RAM, VRAM) sont limitées et/ou quand l'accès à un modèle SaaS par des API de type RESTful n'est pas adapté.

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/algorithms/large-language-models.ipynb)

In [ ]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U torch
# !pip3 install -q -U transformers

# À installer dans tous les cas pour Google Colab et Github
if 'google.colab' in sys.modules or 'CI' in os.environ:
    #!pip3 install -q -U accelerate # requis par Croissant (device_map="auto")
    !pip3 install -q -U sentencepiece # requis par Camembert
    !pip3 install -q -U sacremoses # requis par Flaubert

In [ ]:
# Camembert
camembert_name = "camembert/camembert-base"
# Croissant
# croissant_name = "croissantllm/CroissantLLMBase"
# Flaubert
flaubert_name = "flaubert/flaubert_base_cased"

## Tokenizers

In [ ]:
from transformers import AutoTokenizer, CamembertTokenizer, FlaubertTokenizer

# Camembert
camembert_tokenizer = CamembertTokenizer.from_pretrained(camembert_name)
# Croissant
# croissant_tokenizer = AutoTokenizer.from_pretrained(croissant_name)
# Flaubert
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(flaubert_name, do_lowercase=False)

In [ ]:
print(camembert_tokenizer)

In [ ]:
# print(croissant_tokenizer)

In [ ]:
print(flaubert_tokenizer)

In [ ]:
def check_vocab(name='<s>', tokenizer=camembert_tokenizer):
    vocab = tokenizer.get_vocab()
    try:
        id = vocab[name]
        print(f"{name} est dans le vocabulaire de {tokenizer.name_or_path}: {id}")
    except KeyError:
        print(f"{name} n'est pas dans le vocabulaire de {tokenizer.name_or_path}")

# Camembert
check_vocab()
check_vocab('chat')
check_vocab('chien')
check_vocab('balle')
print()
# Croissant
# check_vocab(tokenizer=croissant_tokenizer)
# check_vocab('chat', croissant_tokenizer)
# check_vocab('chien', croissant_tokenizer)
# check_vocab('balle', croissant_tokenizer)
# print()
# Flaubert
check_vocab(tokenizer=flaubert_tokenizer)
check_vocab('chat', flaubert_tokenizer)
check_vocab('chien', flaubert_tokenizer)
check_vocab('balle', flaubert_tokenizer)

## Embeddings

In [ ]:
import torch
from transformers import AutoModelForCausalLM, CamembertModel, FlaubertModel

# Camembert
camembert_model = CamembertModel.from_pretrained(camembert_name)
# Croissant
# croissant_model = AutoModelForCausalLM.from_pretrained(croissant_name, torch_dtype=torch.float16, device_map="auto")
# Flaubert
flaubert_model = FlaubertModel.from_pretrained(flaubert_name)

## Modèles

In [ ]:
import torch
from transformers import AutoModelForCausalLM, CamembertModel, FlaubertModel

# Camembert
camembert_model = CamembertModel.from_pretrained(camembert_name)
# Croissant
# croissant_model = AutoModelForCausalLM.from_pretrained(croissant_name, torch_dtype=torch.float16, device_map="auto")
# Flaubert
flaubert_model = FlaubertModel.from_pretrained(flaubert_name)

In [ ]:
from transformers import pipeline

nlp_fill_camembert = pipeline('fill-mask', model=camembert_name, top_k=10)
# nlp_fill_croissant = pipeline('fill-mask', model=croissant_name, top_k=10)
nlp_fill_flaubert = pipeline('fill-mask', model=flaubert_name, top_k=10)

In [ ]:
# TODO https://medium.com/@xiaoouwang/playing-with-camembert-and-flaubert-8c5d40e502a7

In [ ]:
# inputs = tokenizer("I am so tired I could sleep right now. -> Je suis si fatigué que je pourrais m'endormir maintenant.\nHe is heading to the market. -> Il va au marché.\nWe are running on the beach. ->", return_tensors="pt").to(model.device)
# tokens = model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60, temperature=0.3)
# print(tokenizer.decode(tokens[0]))

# remove bos token
# inputs = tokenizer("Capitales: France -> Paris, Italie -> Rome, Allemagne -> Berlin, Espagne ->", return_tensors="pt", add_special_tokens=True).to(model.device)
# tokens = model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60)
# print(tokenizer.decode(tokens[0]))

In [ ]:
# Pour supprimer les modèles du cache
# !pip3 install -q -U "huggingface_hub[cli]"
# !huggingface-cli scan-cache
# !huggingface-cli delete-cache